In [1]:
!pip install pyTelegramBotAPI
!pip install transformers

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import telebot

### Подготовка данных
Чистим файл с анекдотами от пустых строк и добавляем теги, на которые будет ориентироваться модель

In [3]:
# Откроем исходный файл и избавимся от пустых строк

anek_path = r"C:\Users\*****\Downloads\Telegram Desktop\anek.txt"
l = []

with open(anek_path, 'r', encoding="utf-8") as anek_text:
    for line in anek_text:
        if line.strip():
            l.append(line.strip())

In [4]:
# Создадим датафрейм на основе получившегося списка анекдотов

df = pd.DataFrame(l)
df.rename(columns={ df.columns[0]: "clean_text"}, inplace = True)
df

,clean_text
0,"Только заметил, что слово ""п@рно"" набирается с..."
1,"Друзья мои, чтобы соответствовать вам, я готов..."
2,"- Люся, ты все еще хранишь мой подарок?- Да.- ..."
3,"- А вот скажи честно, ты во сне храпишь?- Поня..."
4,"Поссорилась с мужем. Пока он спал, я мысленно ..."
...,...
124160,Последним раскрытым громким преступлением в Ки...
124161,Если бесконечное количество российских футболи...
124162,На чемпионат мира по футболу от России нужно Ю...
124163,В целях профилактики от всего весной следует е...


In [5]:
# Добавим к каждому анекдоту тег окончания текста, чтобы модель понимала где обычно они заканчиваются

texts = '<|endoftext|>'.join([text.replace('\n\n\n\n','') for text in df['clean_text']])

In [6]:
# Проверим, как выглядит текст после изменений. Возьмем первые 1000 символов:

texts[:1000]

'Только заметил, что слово "п@рно" набирается самими центральными клавишами. Как все продумано, блин!<|endoftext|>Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!<|endoftext|>- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...<|endoftext|>- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.<|endoftext|>Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.<|endoftext|>Если тебя посещают мысли о смерти - это еще полбеды. Беда - это когда смерть посещают мысли о

In [7]:
# Сохраним наши новые данные в файл

texts_path = 'texts_dataset.txt'
with open(texts_path, "w", encoding='utf-8') as d:
    d.write(texts)

In [8]:
# Посмотрим, что они корректно сохранились

with open(texts_path, "r", encoding='utf-8') as d:
  opened_texts = d.read()

opened_texts[:1000]

'Только заметил, что слово "п@рно" набирается самими центральными клавишами. Как все продумано, блин!<|endoftext|>Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!<|endoftext|>- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...<|endoftext|>- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.<|endoftext|>Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.<|endoftext|>Если тебя посещают мысли о смерти - это еще полбеды. Беда - это когда смерть посещают мысли о

### Запускаем обучение модели
Поскольку бесплатный тариф Google Colab ограничивает доступ к GPU-ресурсам (мы попробовали прогнать medium и large в 3 эпохи, и оба раза заканчивалось хранилище или допустимое время запуска Колаба, и больше нам вообще не дают подключиться к GPU-рантайму), было принято решение обучить модель на своём домашнем ПК. Наша RTX 2060 обладает только 6 ГБ видеопамяти, которых ей тоже не хватает для работы с medium и large, поэтому мы вынуждены пользоваться моделью rugpt3small_based_on_gpt2. Тем не менее итоговые веса все равно позволяют нормально обучить модель в ячейках ниже, и она в результате выдаёт достаточно правдоподобные анекдоты.


In [10]:
# Если планируется воспроизведение, на RTX 2060 6GB три эпохи занимают около часа, вес выходной папки - около 30 ГБ

%%time
!set PYTHONPATH=%PYTHONPATH%;D:\finetuned\ru-gpts
!set CUDA_VISIBLE_DEVICES=0 & python D:\finetuned\ru-gpts\pretrain_transformers.py \
    --output_dir=model_out \
    --model_type=gpt2 \
    --model_name_or_path=D:\sberbank-ai \
    --do_train \
    --train_data_file=D:\finetuned\texts_dataset.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 3 \
    --block_size 1024 \
    --overwrite_output_dir

In [9]:
finetuned_path = 'D:/finetuned/model_out'
model_name_or_path = finetuned_path # Указываем путь до папки с нашей моделью
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path) # Активируем токенизатор
gpt = AutoModelForCausalLM.from_pretrained(model_name_or_path) # Активируем модель

### Генерируем текст
Аргументы для обучения подобраны вручную путём проб и субъективной оценки качества выдаваемых анекдотов. В нынешнем виде анекдоты получаются наиболее складными и натуральными, а иногда даже действительно смешными :)

In [11]:
prompt = 'Студенты ВШЭ' # Начальные слова, которые подаются модели. С них и будет начинаться генерируемый текст
input_ids = tokenizer.encode(prompt, return_tensors="pt") # С помощью токенизатора преобразуем начальные слова в формат для модели

generated_text_samples = gpt.generate(
    input_ids, 
    max_length=100,
    num_return_sequences=1,
    num_beams=2,
    no_repeat_ngram_size=3,
    repetition_penalty=1.5,
    top_p=1.,
    temperature=0.8,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

# Выводим результат генерации
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=False).split('<|endoftext|>')[0]))
  print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Студенты ВШЭ, получив в подарок от Деда Мороза волшебную палочку, на следующий день пошли в военкомат.



### Создаем ТГ Бота
Бот получает от пользователя промпт, на основе которого генерирует анекдот и отправляет его в чат

In [12]:
def model(text, mo):
  input_ids = tokenizer.encode(text, return_tensors="pt")

  generated_text_samples = mo.generate(
      input_ids, 
      max_length=70,
      num_return_sequences=1,
      num_beams=2,
      no_repeat_ngram_size=3,
      repetition_penalty=1.5,
      top_p=1.,
      temperature=0.8,
      do_sample=True,
      top_k=125,
      early_stopping=True
  )
  return tokenizer.decode(generated_text_samples[0], skip_special_tokens=False).split('<|endoftext|>')[0]

teletoken = '' # Вводим наш токен
bot = telebot.TeleBot(teletoken)

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Пожалуйста, напишите начало анекдота')

@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, model(message.text, gpt))
    bot.send_message(message.chat.id, 'Если хотите продолжить, напишите, пожалуйста, начало следующего анекдота')

bot.polling(none_stop=True, interval=0)